## Question 1

The problem: Race condition!
2 threads shared resource counter, and counter += 1 is not an atomic operation - we must lock it

In [ ]:
import threading

counter = 0

lock = threading.Lock() # protect the counter! lock only one resource!

def increment():
  global counter
  for _ in range(1_000_000):
    with lock:          # fiest example using with block
      counter += 1

def decrement():
  global counter
  for _ in range(1_000_000):
    lock.acquire()      # second example with using acquire and release methods
    counter -= 1
    lock.release()

thread1 = threading.Thread(target=increment)
thread2 = threading.Thread(target=decrement)
thread1.start()
thread2.start()


thread1.join()
thread2.join()

print("Final counter value:", counter)

# Another option: using Counter object from collections

Final counter value: 0


## Question 2

In [2]:
import threading

class Bank:

  def __init__(self, balance):
    self._balance = balance
    self._lock = threading.Lock()

  def get_balance(self):
    return self._balance

  def set_balance(self, balance):
    self._balance = balance

  balance = property(get_balance, set_balance)

  def withdraw(self, withdraw):
    with self._lock:
      if (self._balance >= withdraw):
        self._balance -= withdraw
        return True
    return False

  def deposit(self, deposit):
    with self._lock:
      self._balance += deposit

In [3]:
import random
import time

account = Bank(20_000)

def do_deposit(account):
  for _ in range(100):
    account.deposit(random.randint(1, 200_000))

def do_withdraw(account):
  for _ in range(100):
    if(not account.withdraw(random.randint(1, 200_000))):
      time.sleep(0.5)

thread1 = threading.Thread(target=do_deposit, args=(account, ))
thread2 = threading.Thread(target=do_withdraw, args=(account, ))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("Balance: ", account.balance)

Balance:  1150205


## Question 3

In [1]:
import random
import time
from threading import Thread
from queue import Queue

BUFFER_SIZE = 20

In [2]:
# BUFFER - QUEUE

queue = Queue(maxsize=BUFFER_SIZE)

def producer():
  global queue
  counter = 0
  while True:
    item = tuple([random.randint(1, 10) for _ in range (5)])
    queue.put(item) # If queue is full thread waits
    counter += 1
    if (counter % 5 == 0):
      time.sleep(0.5)

def consumer():
  global queue
  dict1 = {}
  counter = 0
  while True:
    item = queue.get() # if queue is empty thread waits
    for n in item:
      if n in dict1.keys():
        dict1[n] += 1
      else:
        dict1[n] = 0
      counter += 1
      if (counter % 100 == 0):
        print(dict1.items())
    time.sleep(1) # simulate some work


producer_thread = Thread(target=producer)
consumer_thread = Thread(target=consumer)
producer_thread.start()
consumer_thread.start()
producer_thread.join() # Only for colab
consumer_thread.join() # Only for colab

dict_items([(5, 4), (9, 14), (8, 11), (3, 10), (10, 10), (1, 11), (6, 14), (4, 5), (7, 4), (2, 7)])
dict_items([(5, 15), (9, 23), (8, 23), (3, 21), (10, 18), (1, 19), (6, 26), (4, 14), (7, 12), (2, 19)])
dict_items([(5, 25), (9, 37), (8, 30), (3, 27), (10, 25), (1, 40), (6, 36), (4, 21), (7, 20), (2, 29)])


KeyboardInterrupt: ignored

In [ ]:
# BUFFER - LIST
buffer_list = []

def producer():
  global buffer_list
  counter = 0
  while True:
    item = tuple([random.randint(1, 100) for _ in range (5)])
    if len(buffer_list) < BUFFER_SIZE:
      buffer_list.append(item)
      counter += 1
    else:
      print("Buffer is full. producer is waiting...")

    if (counter % 5 == 0):
      time.sleep(0.5)


def consumer():
  global buffer_list
  dict1 = {}
  while True:
    if len(buffer_list) > 0:
      item = buffer_list.pop(0)
      for i in item:
        if i in dict1.keys():
          dict1[i] += 1
        else:
          dict1[i] = 0
        counter += 1
        if (counter % 100 == 0):
          print(dict1.items())
        time.sleep(1)

    else:
      # if queue is empty thread waits
      print("Buffer is empty. Consumer is waiting...")

producer_thread = Thread(target=producer)
consumer_thread = Thread(target=consumer)
producer_thread.start()
consumer_thread.start()
producer_thread.join() # Only for colab
consumer_thread.join() # Only for colab

producing (42, 35, 10, 67, 93)
consuming (42, 35, 10, 67, 93)


In [3]:
# 3 producers with queue

# BUFFER - QUEUE

BUFFER_SIZE = 60
queue = Queue(maxsize=BUFFER_SIZE)

def producer():
  global queue
  counter = 0
  while True:
    item = tuple([random.randint(1, 10) for _ in range (5)])
    queue.put(item) # If queue is full thread waits
    counter += 1
    if (counter % 5 == 0):
      time.sleep(0.5)

def consumer():
  global queue
  dict1 = {}
  counter = 0
  while True:
    item = queue.get() # if queue is empty thread waits
    for i in item:
      if i in dict1.keys():
        dict1[i] += 1
      else:
        dict1[i] = 0
      counter += 1
      if (counter % 100 == 0):
        print(dict1.items())
    time.sleep(1) # simulate some work
producers = []
for i in range(3):
  producers.append(Thread(target=producer))
  producers[i].start()

consumer_thread = Thread(target=consumer)
consumer_thread.start()

for i in range(3):
  producers[i].join()

consumer_thread.join() # Only for colab



dict_items([(5, 284), (9, 286), (8, 284), (3, 308), (10, 306), (1, 313), (6, 331), (4, 287), (7, 284), (2, 307)])
dict_items([(4, 17), (7, 9), (1, 12), (3, 5), (10, 9), (2, 8), (9, 6), (6, 12), (8, 6), (5, 6)])
dict_items([(5, 293), (9, 293), (8, 299), (3, 321), (10, 314), (1, 327), (6, 338), (4, 295), (7, 293), (2, 317)])


KeyboardInterrupt: ignored